In [1]:
import pandas as pd
from pathlib import Path

In [2]:
inputfilepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\denmark_spending\data\spending_matrix_full_long_070422.csv")
outputfilepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\denmark_spending\data\spending_matrix_full_long_070422_processed.csv")
region_namespath = Path(r"C:\Users\jasonjia\Dropbox\Projects\denmark_spending\data\dk_municipalities_code_to_name.xlsx")
sector_namespath = Path(r"C:\Users\jasonjia\Dropbox\Projects\denmark_spending\data\sector_number_to_name.xlsx")

In [3]:
# Part 1: Do preliminary processing (mainly change names, reorder columns)

In [4]:
df = pd.read_csv(inputfilepath, low_memory=False)

In [5]:
df.shape

(4618152, 5)

In [6]:
df.head()

,cust_muni,spend_muni,cust_sector,spend_sector,amount_dkk
0,101,101,s01,s01,290048160.0
1,101,101,s01,s02,53024912.0
2,101,101,s01,s03,387442240.0
3,101,101,s01,s04,146490800.0
4,101,101,s01,s05,91945568.0


In [7]:
df.columns

Index(['cust_muni', 'spend_muni', 'cust_sector', 'spend_sector', 'amount_dkk'], dtype='object')

In [8]:
df.rename(columns = {'cust_muni':'spender_region', 
                     'spend_muni':'recipient_region',
                     'cust_sector':'spender_sector',
                     'spend_sector':'recipient_sector',
                     'amount_dkk':'spend'}, inplace = True)

In [9]:
df.head()

,spender_region,recipient_region,spender_sector,recipient_sector,spend
0,101,101,s01,s01,290048160.0
1,101,101,s01,s02,53024912.0
2,101,101,s01,s03,387442240.0
3,101,101,s01,s04,146490800.0
4,101,101,s01,s05,91945568.0


In [10]:
df['spender_sector'] = df['spender_sector'].str.replace('s','')
df['recipient_sector'] = df['recipient_sector'].str.replace('s','')
df

,spender_region,recipient_region,spender_sector,recipient_sector,spend
0,101,101,01,01,2.900482e+08
1,101,101,01,02,5.302491e+07
2,101,101,01,03,3.874422e+08
3,101,101,01,04,1.464908e+08
4,101,101,01,05,9.194557e+07
...,...,...,...,...,...
4618147,860,foreign,28,17,NaN
4618148,860,foreign,28,18,8.696415e+02
4618149,860,foreign,28,77,NaN
4618150,860,foreign,28,88,NaN


In [11]:
df = df.reindex(columns=['spender_region', 'spender_sector', 'recipient_region', 'recipient_sector',
                                 'spend'])
df

,spender_region,spender_sector,recipient_region,recipient_sector,spend
0,101,01,101,01,2.900482e+08
1,101,01,101,02,5.302491e+07
2,101,01,101,03,3.874422e+08
3,101,01,101,04,1.464908e+08
4,101,01,101,05,9.194557e+07
...,...,...,...,...,...
4618147,860,28,foreign,17,NaN
4618148,860,28,foreign,18,8.696415e+02
4618149,860,28,foreign,77,NaN
4618150,860,28,foreign,88,NaN


# Part 2: Convert region codes into actual region names

In [12]:
region_names = pd.read_excel(region_namespath)

In [13]:
region_names.shape

(100, 2)

In [14]:
region_names.head()

,Code,Name
0,101,København
1,147,Frederiksberg
2,151,Ballerup
3,153,Brøndby
4,155,Dragør


In [15]:
# region_names = region_names.append({'Code': 'foreign', 'Name': 'Foreign'}, ignore_index=True) # Already done so in the Excel sheet

In [16]:
region_names

,Code,Name
0,101,København
1,147,Frederiksberg
2,151,Ballerup
3,153,Brøndby
4,155,Dragør
...,...,...
95,846,Mariagerfjord
96,849,Jammerbugt
97,851,Aalborg
98,860,Hjørring


In [17]:
# Because region codes contain numbers (e.g. 101) and strings (i.e. 'foreign'), 
# we need to convert them both to the same data type.
region_names['Code'].dtype

dtype('O')

In [18]:
region_names['Code'] = region_names['Code'].astype('str')
region_names['Code'].dtype

dtype('O')

In [19]:
df['spender_region'] = df['spender_region'].astype('str')
df['spender_region'].dtype

dtype('O')

In [20]:
df_names = df.merge(region_names, left_on='spender_region', right_on='Code', how='left')
df_names

,spender_region,spender_sector,recipient_region,recipient_sector,spend,Code,Name
0,101,01,101,01,2.900482e+08,101,København
1,101,01,101,02,5.302491e+07,101,København
2,101,01,101,03,3.874422e+08,101,København
3,101,01,101,04,1.464908e+08,101,København
4,101,01,101,05,9.194557e+07,101,København
...,...,...,...,...,...,...,...
4618147,860,28,foreign,17,NaN,860,Hjørring
4618148,860,28,foreign,18,8.696415e+02,860,Hjørring
4618149,860,28,foreign,77,NaN,860,Hjørring
4618150,860,28,foreign,88,NaN,860,Hjørring


In [21]:
df_names = df_names.rename(columns = {'Name':'spender_region_name'})
df_names = df_names.drop(['Code'], axis = 1)
df_names

,spender_region,spender_sector,recipient_region,recipient_sector,spend,spender_region_name
0,101,01,101,01,2.900482e+08,København
1,101,01,101,02,5.302491e+07,København
2,101,01,101,03,3.874422e+08,København
3,101,01,101,04,1.464908e+08,København
4,101,01,101,05,9.194557e+07,København
...,...,...,...,...,...,...
4618147,860,28,foreign,17,NaN,Hjørring
4618148,860,28,foreign,18,8.696415e+02,Hjørring
4618149,860,28,foreign,77,NaN,Hjørring
4618150,860,28,foreign,88,NaN,Hjørring


In [22]:
df_names = df_names.merge(region_names, left_on='recipient_region', right_on='Code', how='left')
df_names

,spender_region,spender_sector,recipient_region,recipient_sector,spend,spender_region_name,Code,Name
0,101,01,101,01,2.900482e+08,København,101,København
1,101,01,101,02,5.302491e+07,København,101,København
2,101,01,101,03,3.874422e+08,København,101,København
3,101,01,101,04,1.464908e+08,København,101,København
4,101,01,101,05,9.194557e+07,København,101,København
...,...,...,...,...,...,...,...,...
4618147,860,28,foreign,17,NaN,Hjørring,foreign,Foreign
4618148,860,28,foreign,18,8.696415e+02,Hjørring,foreign,Foreign
4618149,860,28,foreign,77,NaN,Hjørring,foreign,Foreign
4618150,860,28,foreign,88,NaN,Hjørring,foreign,Foreign


In [23]:
df_names = df_names.rename(columns = {'Name':'recipient_region_name'})
df_names = df_names.drop(['Code'], axis = 1)
df_names

,spender_region,spender_sector,recipient_region,recipient_sector,spend,spender_region_name,recipient_region_name
0,101,01,101,01,2.900482e+08,København,København
1,101,01,101,02,5.302491e+07,København,København
2,101,01,101,03,3.874422e+08,København,København
3,101,01,101,04,1.464908e+08,København,København
4,101,01,101,05,9.194557e+07,København,København
...,...,...,...,...,...,...,...
4618147,860,28,foreign,17,NaN,Hjørring,Foreign
4618148,860,28,foreign,18,8.696415e+02,Hjørring,Foreign
4618149,860,28,foreign,77,NaN,Hjørring,Foreign
4618150,860,28,foreign,88,NaN,Hjørring,Foreign


In [24]:
# Test
df_names[(df_names['recipient_sector']=='10') & (df_names['recipient_region']=='846') & (df_names['spender_sector']=='20')] # Test

,spender_region,spender_sector,recipient_region,recipient_sector,spend,spender_region_name,recipient_region_name
33584,101,20,846,10,NaN,København,Mariagerfjord
80708,147,20,846,10,NaN,Frederiksberg,Mariagerfjord
127832,151,20,846,10,NaN,Ballerup,Mariagerfjord
174956,153,20,846,10,NaN,Brøndby,Mariagerfjord
222080,155,20,846,10,NaN,Dragør,Mariagerfjord
...,...,...,...,...,...,...,...
4416116,840,20,846,10,NaN,Rebild,Mariagerfjord
4463240,846,20,846,10,748241.000,Mariagerfjord,Mariagerfjord
4510364,849,20,846,10,NaN,Jammerbugt,Mariagerfjord
4557488,851,20,846,10,17.514,Aalborg,Mariagerfjord


## Skip Part 3 because it creates a lot of lag (filtering through strings vs numbers) -> jump straight to Part 4

In [26]:
# Part 3: Convert sector numbers into actual sector names 
# (But for historical reasons, we drop the original sector numbers, and rename sector_name -> sector)
# (So that no adjustments need to be made on Tableau other than updating the data source)

# Tried but this caused huge lags - trying not to use it for now

In [27]:
sector_names = pd.read_excel(sector_namespath)
sector_names.shape

(28, 2)

In [28]:
sector_names.head()

,sector_number,sector_name
0,1,1 - Food away from home
1,2,2 - Entertainment
2,3,3 - Groceries
3,4,"4 - Personal services, pharmacies"
4,5,"5 - Cars, fuel, car repair, public transport"


In [29]:
# Because sector codes are numbers, but may be viewed as numbers (int64) or strings,
# we need to convert them to the same data type

In [30]:
sector_names['sector_number'].dtype

dtype('int64')

In [31]:
print(df_names['spender_sector'].dtype)
df_names['spender_sector'] = df_names['spender_sector'].astype('int64')
print(df_names['spender_sector'].dtype)

object
int64


In [32]:
print(df_names['recipient_sector'].dtype)
df_names['recipient_sector'] = df_names['recipient_sector'].astype('int64')
print(df_names['recipient_sector'].dtype)

object
int64


In [33]:
# Merge spender sector
df_names = df_names.merge(sector_names, left_on='spender_sector', right_on='sector_number', how='left')
df_names

,spender_region,spender_sector,recipient_region,recipient_sector,spend,spender_region_name,recipient_region_name,sector_number,sector_name
0,101,1,101,1,50520576.0,København,København,1,1 - Food away from home
1,101,1,101,2,8743208.0,København,København,1,1 - Food away from home
2,101,1,101,3,64380656.0,København,København,1,1 - Food away from home
3,101,1,101,4,24944372.0,København,København,1,1 - Food away from home
4,101,1,101,5,15440209.0,København,København,1,1 - Food away from home
...,...,...,...,...,...,...,...,...,...
3841987,foreign,28,foreign,10,NaN,Foreign,Foreign,28,28 - Unemployed
3841988,foreign,28,foreign,11,NaN,Foreign,Foreign,28,28 - Unemployed
3841989,foreign,28,foreign,12,NaN,Foreign,Foreign,28,28 - Unemployed
3841990,foreign,28,foreign,17,NaN,Foreign,Foreign,28,28 - Unemployed


In [34]:
df_names = df_names.drop(['sector_number', 'spender_sector'], axis = 1)
df_names = df_names.rename(columns = {'sector_name':'spender_sector'})
df_names

,spender_region,recipient_region,recipient_sector,spend,spender_region_name,recipient_region_name,spender_sector
0,101,101,1,50520576.0,København,København,1 - Food away from home
1,101,101,2,8743208.0,København,København,1 - Food away from home
2,101,101,3,64380656.0,København,København,1 - Food away from home
3,101,101,4,24944372.0,København,København,1 - Food away from home
4,101,101,5,15440209.0,København,København,1 - Food away from home
...,...,...,...,...,...,...,...
3841987,foreign,foreign,10,NaN,Foreign,Foreign,28 - Unemployed
3841988,foreign,foreign,11,NaN,Foreign,Foreign,28 - Unemployed
3841989,foreign,foreign,12,NaN,Foreign,Foreign,28 - Unemployed
3841990,foreign,foreign,17,NaN,Foreign,Foreign,28 - Unemployed


In [35]:
# Merge recipient sector
df_names = df_names.merge(sector_names, left_on='recipient_sector', right_on='sector_number', how='left')
df_names

,spender_region,recipient_region,recipient_sector,spend,spender_region_name,recipient_region_name,spender_sector,sector_number,sector_name
0,101,101,1,50520576.0,København,København,1 - Food away from home,1,1 - Food away from home
1,101,101,2,8743208.0,København,København,1 - Food away from home,2,2 - Entertainment
2,101,101,3,64380656.0,København,København,1 - Food away from home,3,3 - Groceries
3,101,101,4,24944372.0,København,København,1 - Food away from home,4,"4 - Personal services, pharmacies"
4,101,101,5,15440209.0,København,København,1 - Food away from home,5,"5 - Cars, fuel, car repair, public transport"
...,...,...,...,...,...,...,...,...,...
3841987,foreign,foreign,10,NaN,Foreign,Foreign,28 - Unemployed,10,10 - Utilities
3841988,foreign,foreign,11,NaN,Foreign,Foreign,28 - Unemployed,11,11 - Specialized retail stores
3841989,foreign,foreign,12,NaN,Foreign,Foreign,28 - Unemployed,12,12 - Home improvement
3841990,foreign,foreign,17,NaN,Foreign,Foreign,28 - Unemployed,17,17 - Finance


In [36]:
df_names = df_names.drop(['sector_number', 'recipient_sector'], axis = 1)
df_names = df_names.rename(columns = {'sector_name':'recipient_sector'})
df_names

,spender_region,recipient_region,spend,spender_region_name,recipient_region_name,spender_sector,recipient_sector
0,101,101,50520576.0,København,København,1 - Food away from home,1 - Food away from home
1,101,101,8743208.0,København,København,1 - Food away from home,2 - Entertainment
2,101,101,64380656.0,København,København,1 - Food away from home,3 - Groceries
3,101,101,24944372.0,København,København,1 - Food away from home,"4 - Personal services, pharmacies"
4,101,101,15440209.0,København,København,1 - Food away from home,"5 - Cars, fuel, car repair, public transport"
...,...,...,...,...,...,...,...
3841987,foreign,foreign,NaN,Foreign,Foreign,28 - Unemployed,10 - Utilities
3841988,foreign,foreign,NaN,Foreign,Foreign,28 - Unemployed,11 - Specialized retail stores
3841989,foreign,foreign,NaN,Foreign,Foreign,28 - Unemployed,12 - Home improvement
3841990,foreign,foreign,NaN,Foreign,Foreign,28 - Unemployed,17 - Finance


# Part 4: Reindex columns and save output to csv

In [25]:
df_names = df_names.reindex(columns=['spender_region', 'spender_region_name', 'spender_sector', 
                                 'recipient_region', 'recipient_region_name', 'recipient_sector', 'spend'])
df_names

,spender_region,spender_region_name,spender_sector,recipient_region,recipient_region_name,recipient_sector,spend
0,101,København,01,101,København,01,2.900482e+08
1,101,København,01,101,København,02,5.302491e+07
2,101,København,01,101,København,03,3.874422e+08
3,101,København,01,101,København,04,1.464908e+08
4,101,København,01,101,København,05,9.194557e+07
...,...,...,...,...,...,...,...
4618147,860,Hjørring,28,foreign,Foreign,17,NaN
4618148,860,Hjørring,28,foreign,Foreign,18,8.696415e+02
4618149,860,Hjørring,28,foreign,Foreign,77,NaN
4618150,860,Hjørring,28,foreign,Foreign,88,NaN


In [26]:
df_names.to_csv(outputfilepath, index=False)